In [2]:
%cd "~/pln-brca-xp/notebooks"
%load_ext autoreload
%autoreload 2

/home/xabush/pln-brca-xp/notebooks


In [3]:
import pickle

from goatools.associations import dnld_assc
from goatools.obo_parser import GODag
from goatools.semantic import *

from convert_symbol_to_entrez import read_symbol2geneid
from utils import *
from utils_emb import *

In [4]:
feats_diff = load_features("/var/www/datasets/filteredGenes4studies.txt")
non_embedded = load_features("datasets/non_embedded_genes_list.txt")
feats_diff_filtered = list(set(feats_diff) - set(non_embedded))
len(feats_diff_filtered)

4382

In [5]:
godag = GODag("go-basic.obo")
associations = dnld_assc("datasets/goa_human.gaf", godag, namespace="all")
termcounts = TermCounts(godag, associations)

go-basic.obo: fmt(1.2) rel(2021-02-01) 47,291 GO Terms
HMS:0:00:11.176663 479,438 annotations READ: datasets/goa_human.gaf 
19783 IDs in loaded association branch, all


In [6]:
pln_go_df = pd.read_csv("datasets/pln_goa_comp/goa_pln_diff_2.csv")
goa_df = pd.read_csv("datasets/pln_goa_comp/goa_pln_diff_1.csv")
pln_lst = pln_go_df[pln_go_df["NS"] == "BP"]["# GO"].to_list()
goa_lst = goa_df[goa_df["NS"] == "BP"]["# GO"].to_list()

In [7]:
with open("datasets/pubmed_gene_ann.pickle", "rb") as fp:
    pubmed_gene_ann = pickle.load(fp)

In [8]:
pubmed_genes = []

for k, v in pubmed_gene_ann.items():
    pubmed_genes.extend(v)

pubmed_genes = set(pubmed_genes)
len(pubmed_genes)

1617

In [9]:
sym2geneid = read_symbol2geneid("datasets/Homo_sapiens.gene_info")

  126,696 READ: datasets/Homo_sapiens.gene_info


In [10]:
pop_ids = convert_symbol_to_geneid(feats_diff_filtered ,sym2geneid)
pubmed_intr = list(pubmed_genes & set(pop_ids))
len(pubmed_intr)

530

In [11]:
pubmed_go_df = run_gene_enrich(pubmed_intr, pop_ids, "datasets/pln_goa_comp/list_4_pubmed" ,sym2geneid, convert_study=False, convert_pop=False)
display(pubmed_go_df)

datasets/go-basic.obo: fmt(1.2) rel(2021-02-01) 47,291 GO Terms; optional_attrs(relationship)
HMS:0:00:05.084366 335,863 annotations, 20,672 genes, 18,441 GOs, 1 taxids READ: gene2go 
Study: 530 vs. Population 4382


Load BP Gene Ontology Analysis ...
fisher module not installed.  Falling back on scipy.stats.fisher_exact
Propagating term counts up: is_a
 97%  4,250 of  4,382 population items found in association

Load CC Gene Ontology Analysis ...
fisher module not installed.  Falling back on scipy.stats.fisher_exact
Propagating term counts up: is_a
 99%  4,328 of  4,382 population items found in association

Load MF Gene Ontology Analysis ...
fisher module not installed.  Falling back on scipy.stats.fisher_exact
Propagating term counts up: is_a
 98%  4,296 of  4,382 population items found in association

Run BP Gene Ontology Analysis: current study set of 530 IDs ... 99%    527 of    530 study items found in association
100%    530 of    530 study items found in population(4382)
Calcu

,# GO,NS,enrichment,name,ratio_in_study,ratio_in_pop,p_uncorrected,depth,study_count,p_bonferroni,p_sidak,p_holm,p_fdr_bh,study_items
0,GO:0010941,BP,e,regulation of cell death,189/530,608/4382,5.337717e-43,4,189,6.220576e-39,6.063745e-39,6.220576e-39,6.220576e-39,"142, 183, 207, 220, 292, 301, 308, 332, 367, 3..."
1,GO:0043067,BP,e,regulation of programmed cell death,180/530,562/4382,1.082218e-42,5,180,1.261217e-38,1.229420e-38,1.261109e-38,6.306087e-39,"142, 183, 207, 220, 292, 301, 308, 332, 367, 3..."
2,GO:0009893,BP,e,positive regulation of metabolic process,290/530,1242/4382,2.430488e-42,4,290,2.832491e-38,2.761079e-38,2.832004e-38,9.441635e-39,"47, 142, 183, 196, 199, 207, 301, 367, 374, 38..."
3,GO:0048518,BP,e,positive regulation of biological process,378/530,1923/4382,5.160696e-42,3,378,6.014275e-38,5.862646e-38,6.012727e-38,1.503569e-38,"47, 142, 183, 196, 199, 207, 220, 292, 301, 33..."
4,GO:0048522,BP,e,positive regulation of cellular process,360/530,1780/4382,1.037532e-41,4,360,1.209140e-37,1.178656e-37,1.208725e-37,2.418280e-38,"47, 142, 183, 196, 199, 207, 220, 292, 301, 33..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3199,GO:0004553,MF,p,"hydrolase activity, hydrolyzing O-glycosyl com...",0/530,29/4382,4.196005e-02,4,0,1.000000e+00,1.000000e+00,1.000000e+00,4.001862e-01,NaN
3200,GO:0050136,MF,p,NADH dehydrogenase (quinone) activity,0/530,29/4382,4.196005e-02,6,0,1.000000e+00,1.000000e+00,1.000000e+00,4.001862e-01,NaN
3201,GO:0003954,MF,p,NADH dehydrogenase activity,0/530,29/4382,4.196005e-02,4,0,1.000000e+00,1.000000e+00,1.000000e+00,4.001862e-01,NaN
3202,GO:0008137,MF,p,NADH dehydrogenase (ubiquinone) activity,0/530,29/4382,4.196005e-02,7,0,1.000000e+00,1.000000e+00,1.000000e+00,4.001862e-01,NaN


In [12]:
pubmed_go_bp = pubmed_go_df[pubmed_go_df["NS"] == "BP"]["# GO"].to_list()
len(pubmed_go_bp)

2687

In [13]:
overlap_pln, _, _ = find_overlap_go(pln_lst, pubmed_go_bp)

Num overlap: 184
Num found in list 1, not in list 2:221
Num found in list 2, not in list 1: 2503


In [14]:
overlap_goa, _, _ = find_overlap_go(goa_lst, pubmed_go_bp)

Num overlap: 98
Num found in list 1, not in list 2:95
Num found in list 2, not in list 1: 2589


In [179]:
pln_goa_ss_pubmed = semantic_sim_matrix(pln_lst, pubmed_go_bp, godag, termcounts)
pln_goa_ss_pubmed_df = pd.DataFrame(pln_goa_ss_pubmed, index=pln_lst, columns=pubmed_go_bp)
display(pln_goa_ss_pubmed_df)

,GO:0010941,GO:0043067,GO:0009893,GO:0048518,GO:0048522,GO:0042981,GO:0050793,GO:0010604,GO:0042127,GO:0042221,...,GO:0006906,GO:0071705,GO:0043603,GO:0006811,GO:0043604,GO:0072657,GO:0031123,GO:0006413,GO:0006414,GO:0006820
GO:1902600,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.028672,0.399393,0.036472,0.548096,0.034088,0.035287,0.030648,0.030886,0.030370,0.528623
GO:0098662,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.033035,0.495036,0.043836,0.691587,0.040438,0.042137,0.035687,0.036009,0.035310,0.660870
GO:0070126,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.224725,0.000000,0.034415,0.000000,0.032285,0.033359,0.029183,0.029398,0.028931,0.000000
GO:0019646,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.024081,0.000000,0.171337,0.000000,0.162208,0.028581,0.148612,0.149566,0.147489,0.000000
GO:0006123,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.024081,0.000000,0.171337,0.000000,0.162208,0.028581,0.148612,0.149566,0.147489,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GO:0031124,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.027089,0.000000,0.384734,0.000000,0.361224,0.032920,0.952521,0.169605,0.204855,0.000000
GO:1901362,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.254451,0.000000,0.593739,0.000000,0.455614,0.204136,0.516542,0.000000
GO:0006950,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.619789,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
GO:0009150,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.030913,0.000000,0.455330,0.000000,0.422766,0.038745,0.416710,0.195556,0.181130,0.000000


In [196]:
pln_pubmed = {}

for i, go_1 in enumerate(pln_goa_ss_pubmed_df.index.to_list()):
    pln_pubmed[go_1] = []
    for j, go_2 in enumerate(pln_goa_ss_pubmed_df.columns.to_list()):
        if not np.isnan(pln_goa_ss_pubmed_df.iat[i, j]) and pln_goa_ss_pubmed_df.iat[i, j] >= 0.8:
            pln_pubmed[go_1].append(go_2)

In [1]:
list(pln_pubmed.items())[:5]

NameError: name 'pln_pubmed' is not defined

In [197]:
pln_pubmed_gos = []
for k, v in  pln_pubmed.items():
    if len(v) > 0:
        pln_pubmed_gos.extend(v)

pln_pubmed_gos = set(pln_pubmed_gos)
len(pln_pubmed_gos)

551

In [198]:
pln_pubmed_go_df = pubmed_go_df[pubmed_go_df["# GO"].isin(pln_pubmed_gos)]
pln_pubmed_go_df.set_index("# GO", inplace=True)
display(pln_pubmed_go_df)

,NS,enrichment,name,ratio_in_study,ratio_in_pop,p_uncorrected,depth,study_count,p_bonferroni,p_sidak,p_holm,p_fdr_bh,study_items
# GO,,,,,,,,,,,,,
GO:0010941,BP,e,regulation of cell death,189/530,608/4382,5.337717e-43,4,189,6.220576e-39,6.063745e-39,6.220576e-39,6.220576e-39,"142, 183, 207, 220, 292, 301, 308, 332, 367, 3..."
GO:0043067,BP,e,regulation of programmed cell death,180/530,562/4382,1.082218e-42,5,180,1.261217e-38,1.229420e-38,1.261109e-38,6.306087e-39,"142, 183, 207, 220, 292, 301, 308, 332, 367, 3..."
GO:0009893,BP,e,positive regulation of metabolic process,290/530,1242/4382,2.430488e-42,4,290,2.832491e-38,2.761079e-38,2.832004e-38,9.441635e-39,"47, 142, 183, 196, 199, 207, 301, 367, 374, 38..."
GO:0048518,BP,e,positive regulation of biological process,378/530,1923/4382,5.160696e-42,3,378,6.014275e-38,5.862646e-38,6.012727e-38,1.503569e-38,"47, 142, 183, 196, 199, 207, 220, 292, 301, 33..."
GO:0048522,BP,e,positive regulation of cellular process,360/530,1780/4382,1.037532e-41,4,360,1.209140e-37,1.178656e-37,1.208725e-37,2.418280e-38,"47, 142, 183, 196, 199, 207, 220, 292, 301, 33..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
GO:0072657,BP,p,protein localization to membrane,15/530,203/4382,3.570039e-02,5,15,1.000000e+00,1.000000e+00,1.000000e+00,1.751800e-01,"999, 1020, 2316, 3551, 5590, 5906, 6386, 7074,..."
GO:0031123,BP,p,RNA 3'-end processing,1/530,46/4382,3.760717e-02,8,1,1.000000e+00,1.000000e+00,1.000000e+00,1.836771e-01,6741
GO:0006413,BP,p,translational initiation,4/530,82/4382,3.989929e-02,3,4,1.000000e+00,1.000000e+00,1.000000e+00,1.836771e-01,"1968, 1977, 7311, 8894"


In [136]:
def calc_overlap(s1, s2):
    intr = len(s1 & s2)
    return intr / (len(s1) + len(s2) - intr)

In [199]:
pubmed_id_gos_pln = {}

for pubmed_id in pubmed_gene_ann:
    pubmed_id_gos_pln[pubmed_id] = []
    for go in pln_pubmed_go_df.index:
        study_item_ls = pln_pubmed_go_df.loc[go]["study_items"]
        #check if we got a nan value
        if isinstance(study_item_ls, float): continue
        study_item_ls = study_item_ls.split(",")
        study_items = [int(x.strip()) for x in study_item_ls]
        overlap_ratio = calc_overlap(pubmed_gene_ann[pubmed_id], set(study_items))
        if overlap_ratio > 0.2:
            pubmed_id_gos_pln[pubmed_id].append(go)

    if len(pubmed_id_gos_pln[pubmed_id]) == 0:
        del pubmed_id_gos_pln[pubmed_id]

len(pubmed_id_gos_pln)

253

In [31]:
list(pubmed_id_gos_pln.items())[0]

('33993438',
 ['GO:0016310',
  'GO:0043412',
  'GO:0009891',
  'GO:0010557',
  'GO:0051179',
  'GO:0010608',
  'GO:0035556',
  'GO:0006464',
  'GO:0010629',
  'GO:0048732',
  'GO:0002684',
  'GO:0031328',
  'GO:0002768',
  'GO:0006807',
  'GO:0048584',
  'GO:0043086',
  'GO:0048522',
  'GO:0048639',
  'GO:0043067',
  'GO:0036211',
  'GO:0031100',
  'GO:0048513',
  'GO:0046822',
  'GO:0038093',
  'GO:0065003',
  'GO:1901360',
  'GO:0044419',
  'GO:1903047',
  'GO:0002757',
  'GO:1901564',
  'GO:0009719',
  'GO:0070646',
  'GO:0000226',
  'GO:0006796',
  'GO:0065008',
  'GO:0044238',
  'GO:1901698',
  'GO:0048583',
  'GO:0008152',
  'GO:0016579',
  'GO:0007051',
  'GO:0043170',
  'GO:0042326',
  'GO:0009987',
  'GO:0008150',
  'GO:0050896',
  'GO:1902680',
  'GO:0044260',
  'GO:0002429',
  'GO:0045893',
  'GO:0009725',
  'GO:0042221',
  'GO:0044267',
  'GO:0036294',
  'GO:0010033',
  'GO:0048518',
  'GO:0010941',
  'GO:0006468',
  'GO:0071704',
  'GO:0045936',
  'GO:0002764',
  'GO:00380

In [201]:
goa_ss_pubmed = semantic_sim_matrix(goa_lst, pubmed_go_bp, godag, termcounts)
goa_ss_pubmed_df = pd.DataFrame(goa_ss_pubmed, index=goa_lst, columns=pubmed_go_bp)
display(goa_ss_pubmed_df)

,GO:0010941,GO:0043067,GO:0009893,GO:0048518,GO:0048522,GO:0042981,GO:0050793,GO:0010604,GO:0042127,GO:0042221,...,GO:0006906,GO:0071705,GO:0043603,GO:0006811,GO:0043604,GO:0072657,GO:0031123,GO:0006413,GO:0006414,GO:0006820
GO:0006658,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.024965,0.0,0.179069,0.0,0.169122,0.029836,0.145638,0.155425,0.144495,0.0
GO:0070986,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
GO:0002064,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.028069,0.0,0.035502,0.0,0.033239,0.034379,0.029961,0.030187,0.029695,0.0
GO:0030512,0.118521,0.117319,0.119031,0.128641,0.140234,0.117000,0.112418,0.117813,0.118135,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
GO:0051965,0.117245,0.116069,0.299284,0.323135,0.347723,0.115756,0.532223,0.296257,0.116867,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GO:0001558,0.151445,0.149488,0.156950,0.174099,0.188798,0.148970,0.145652,0.154839,0.150815,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
GO:0010948,0.146086,0.144265,0.150615,0.166338,0.180542,0.143782,0.140180,0.148670,0.145500,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
GO:0002683,0.134769,0.133054,0.154204,0.170726,0.167377,0.132599,0.143283,0.152165,0.134216,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
GO:0001818,0.128202,0.126650,0.338468,0.160323,0.157367,0.126238,0.135884,0.360780,0.127702,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


In [202]:
goa_pubmed = {}

for i, go_1 in enumerate(goa_ss_pubmed_df.index.to_list()):
    goa_pubmed[go_1] = []
    for j, go_2 in enumerate(goa_ss_pubmed_df.columns.to_list()):
        if not np.isnan(goa_ss_pubmed_df.iat[i, j]) and goa_ss_pubmed_df.iat[i, j] >= 0.8:
            goa_pubmed[go_1].append(go_2)

In [204]:
goa_pubmed_gos = []
for k, v in  goa_pubmed.items():
    if len(v) > 0:
        goa_pubmed_gos.extend(v)

goa_pubmed_gos = set(goa_pubmed_gos)
len(goa_pubmed_gos)

411

In [205]:
goa_pubmed_go_df = pubmed_go_df[pubmed_go_df["# GO"].isin(goa_pubmed_gos)]
goa_pubmed_go_df.set_index("# GO", inplace=True)
display(goa_pubmed_go_df)

,NS,enrichment,name,ratio_in_study,ratio_in_pop,p_uncorrected,depth,study_count,p_bonferroni,p_sidak,p_holm,p_fdr_bh,study_items
# GO,,,,,,,,,,,,,
GO:0048523,BP,e,negative regulation of cellular process,307/530,1524/4382,4.015752e-31,4,307,4.679957e-27,4.561968e-27,4.671122e-27,2.034764e-28,"142, 183, 196, 199, 207, 292, 301, 308, 332, 3..."
GO:0048519,BP,e,negative regulation of biological process,325/530,1700/4382,6.059478e-29,3,325,7.061716e-25,6.883679e-25,7.044750e-25,2.435075e-26,"142, 183, 196, 199, 207, 216, 292, 301, 308, 3..."
GO:0048584,BP,e,positive regulation of response to stimulus,189/530,748/4382,9.136457e-29,4,189,1.064763e-24,1.037918e-24,1.062022e-24,3.434718e-26,"142, 183, 199, 207, 301, 367, 374, 387, 389, 4..."
GO:0001932,BP,e,regulation of protein phosphorylation,127/530,401/4382,2.325855e-28,8,127,2.710551e-24,2.642214e-24,2.703108e-24,8.213791e-26,"183, 207, 374, 387, 472, 595, 836, 890, 891, 9..."
GO:1902531,BP,e,regulation of intracellular signal transduction,164/530,605/4382,2.839704e-28,6,164,3.309392e-24,3.225957e-24,3.300020e-24,9.566656e-26,"142, 183, 207, 367, 374, 387, 389, 396, 467, 4..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
GO:0002705,BP,e,positive regulation of leukocyte mediated immu...,8/530,31/4382,4.547103e-02,6,8,1.000000e+00,1.000000e+00,1.000000e+00,1.997734e-01,"2625, 3554, 3684, 4292, 4436, 5590, 7158, 22890"
GO:2000272,BP,e,negative regulation of signaling receptor acti...,4/530,12/4382,4.735140e-02,7,4,1.000000e+00,1.000000e+00,1.000000e+00,2.058311e-01,"4043, 5728, 9052, 27122"
GO:0030316,BP,e,osteoclast differentiation,4/530,12/4382,4.735140e-02,6,4,1.000000e+00,1.000000e+00,1.000000e+00,2.058311e-01,"1432, 1436, 3726, 54209"


In [206]:
pubmed_id_gos_goa = {}

for pubmed_id in pubmed_gene_ann:
    pubmed_id_gos_goa[pubmed_id] = []
    for go in overlap_goa:
        study_item_ls = goa_pubmed_go_df.loc[go]["study_items"]
        #check if we got a nan value
        if isinstance(study_item_ls, float): continue
        study_item_ls = study_item_ls.split(",")
        study_items = [int(x.strip()) for x in study_item_ls]
        overlap_ratio = calc_overlap(pubmed_gene_ann[pubmed_id], set(study_items))
        if overlap_ratio > 0.2:
            pubmed_id_gos_goa[pubmed_id].append(go)

    if len(pubmed_id_gos_goa[pubmed_id]) == 0:
        del pubmed_id_gos_goa[pubmed_id]

len(pubmed_id_gos_goa)

25

In [207]:
pubmed_id_pln = [k.strip() for k in pubmed_id_gos_pln]
pubmed_id_goa = [k.strip() for k in pubmed_id_gos_goa]
print(len(pubmed_id_pln))
print(len(pubmed_id_goa))

253
25


In [209]:
pln_goa_pubmed_diff = list(set(pubmed_id_pln) - set(pubmed_id_goa))
print(len(pln_goa_pubmed_diff))
print(pln_goa_pubmed_diff)
goa_pln_pubmed_diff = list(set(pubmed_id_goa) - set(pubmed_id_pln))
print(len(goa_pln_pubmed_diff))
print(goa_pln_pubmed_diff)


246
['33855848', '31548345', '32789819', '33896818', '33925616', '31422497', '33627502', '33721131', '33636580', '31496995', '31463793', '31167163', '30547831', '29804556', '32312891', '33963381', '32370060', '31222709', '32727783', '32368395', '31871111', '32081423', '31439579', '32142667', '31146345', '29522117', '30515568', '30560461', '32990543', '31578396', '31782270', '32926435', '32684154', '30911948', '30182339', '33670518', '32788222', '33869016', '31327134', '31647985', '33660534', '34016642', '33809171', '32326116', '29555554', '32362585', '33849913', '31280452', '31385051', '33318536', '32298374', '33613637', '32560703', '33982878', '33996588', '32826278', '33840002', '33658859', '31602271', '31216817', '33738261', '33791875', '33452979', '32014063', '31195751', '31501276', '31891878', '32526980', '29433542', '31588558', '31917424', '33117280', '30431262', '31002529', '33440675', '32330493', '33566819', '32963604', '31281728', '30191958', '31307407', '32014905', '32108307',